In [2]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
import transformers
from transformers import Trainer, AutoModel, AutoTokenizer, AutoModelForSequenceClassification, set_seed
from transformers import TrainingArguments, EarlyStoppingCallback, BioGptForSequenceClassification, get_scheduler
import datasets
from datasets import Dataset, load_dataset, load_from_disk
import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.utils.checkpoint
import json
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import scipy
import wandb

from torch import cuda
DEVICE = torch.device('cuda' if cuda.is_available() else 'cpu')
device = torch.device('cuda' if cuda.is_available() else 'cpu')

In [3]:
random_state = 42
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)
set_seed(42)

In [4]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joemenke (nlp4health). Use `wandb login --relogin` to force relogin


True

## Data Loading

In [5]:
df = pd.read_csv('data/doc_summary_predictions.csv')

# Extracted text
def_df = df[df['prediction'] == 1]
def_df = def_df[['text', 'name']]
def_df['name'] = def_df['name'].apply(lambda s: s.replace('_', ' ').replace('.txt', ''))
def_df = def_df.groupby('name')['text'].apply(' '.join).reset_index()
def_df = def_df[['name', 'text']]

# Meta-Analysis Labels
label_df = pd.read_csv('data/labels_lenient.csv')
label_df['Study'] = label_df['Study'].apply(lambda s: s.replace('_', ' ').replace('.txt', ''))

# Merge text and labels
label_df.rename(columns = {'Study':'name'}, inplace = True)
merged_df = def_df.merge(label_df, how='inner', on='name')

# Generate list of labels (igt, ifg-ada, ifg-who, hba1c-ada, hba1c-iec)
merged_df['labels'] = merged_df[merged_df.columns[2:]].values.tolist()
merged_df = merged_df.drop(columns=['gold_igt', 'gold_ifg_ada', 'gold_ifg_who', 'gold_hba1c_ada', 'gold_hba1c_iec']).reset_index(drop=True)
merged_df

,name,text,labels
0,Ahn 2018,The risk for coronary heart disease in women s...,"[0, 1, 1, 0, 0]"
1,Ares 2019,Age- and sex-stratified hazard ratios (HRs) we...,"[1, 1, 0, 1, 0]"
2,Barr 2007,Known diabetes mellitus (HR 2.6 95% CI 1.4 to ...,"[1, 0, 1, 0, 0]"
3,Barzilay 1999,The new fasting American Diabetes Association ...,"[1, 1, 1, 0, 0]"
4,Bergman 2015,Prediabetes comprising impaired fasting glucos...,"[1, 1, 0, 0, 0]"
...,...,...,...
125,Wen 2005,RESULTS: FBG > or =110 mg/dl was associated wi...,"[0, 1, 1, 0, 0]"
126,Wild 2005,RESULTS: Prevalence of undiagnosed diabetes us...,"[1, 0, 1, 0, 0]"
127,Yeboah 2011,Type 2 DM was defined as fasting glucose >125 ...,"[0, 1, 0, 0, 0]"
128,Zhang 2008,Hypertension was defined by the criteria of th...,"[1, 0, 1, 0, 0]"


## Model Development 

In [6]:
def stratify_multilabel_text(data_df, test_size):
    """
    Inputs:
        data_df: DataFrame with 'text' and 'labels' columns
        test_size: [0,1]
    Outputs:
        train and test DataFrames with 'text' and 'labels' columns
    """
    
    data_df = data_df.reset_index()
    data_df['index'] = data_df['index'].apply(lambda s: [s])
    X = scipy.sparse.lil_matrix(data_df['index'].to_list())
    y = scipy.sparse.lil_matrix(data_df['labels'].to_list())
    
    X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size=test_size)
    
    y_train = pd.Series(pd.DataFrame.sparse.from_spmatrix(y_train).values.tolist()).rename('labels')
    y_test = pd.Series(pd.DataFrame.sparse.from_spmatrix(y_test).values.tolist()).rename('labels')
    
    text_dict = data_df.to_dict()['text']
    X_train = pd.DataFrame.sparse.from_spmatrix(X_train)[0].map(text_dict).rename('text')
    X_test = pd.DataFrame.sparse.from_spmatrix(X_test)[0].map(text_dict).rename('text')
    
    train = pd.concat([X_train, y_train], axis=1)
    test = pd.concat([X_test, y_test], axis=1)
    
    return train, test

In [7]:
def preprocessing(dataset, tokenizer, max_length):
    dataset = datasets.Dataset.from_pandas(dataset[['text', 'labels']], preserve_index=False)
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=max_length)
    dataset_token = dataset.map(tokenize_function)
    return dataset_token

class MultiLabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop('labels') #keeps the labels
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), 
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss

def train_huggingface_model(output_dir, model_name, tokenizer, train_dataset, dev_dataset, num_labels, num_epochs, learning_rate, weight_decay, early_stop_var, grad_steps):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, 
        num_labels=num_labels,
        problem_type="multi_label_classification",
    ).to(device)
    model.gradient_checkpointing_enable()
    
    early_stop = EarlyStoppingCallback(early_stopping_patience = early_stop_var)
    training_args = TrainingArguments(
        output_dir = output_dir,
        evaluation_strategy = "steps",
        logging_strategy = "steps",
        logging_steps = STEPS,
        eval_steps = STEPS,
        per_device_train_batch_size = 1,
        per_device_eval_batch_size = 1,
        gradient_accumulation_steps = grad_steps,
        gradient_checkpointing = True,
        num_train_epochs = num_epochs,
        learning_rate = learning_rate,
        weight_decay = weight_decay,
        load_best_model_at_end = True,
        metric_for_best_model = 'macro F1',
        report_to = "wandb",
    )
    
    def compute_metrics(eval_preds):
        all_labels = ['IGT', 'IFG_ADA', 'IFG_WHO', 'HbA1c_ADA', 'HbA1c_IEC']
        predictions, labels = eval_preds
        y_pred = torch.sigmoid(torch.from_numpy(predictions)) 
        y_pred = (y_pred.numpy()>0.5).astype(float)
        clf_dict = metrics.classification_report(labels, y_pred, target_names=all_labels,
                                         zero_division=0, output_dict=True)
        return {
            "IGT P": clf_dict['IGT']['precision'],
            "IGT R": clf_dict['IGT']['recall'],
            "IGT F1": clf_dict['IGT']['f1-score'],
            "IFG_ADA P": clf_dict['IFG_ADA']['precision'],
            "IFG_ADA R": clf_dict['IFG_ADA']['recall'],
            "IFG_ADA F1": clf_dict['IFG_ADA']['f1-score'], 
            "IFG_WHO P": clf_dict['IFG_WHO']['precision'],
            "IFG_WHO R": clf_dict['IFG_WHO']['recall'],
            "IFG_WHO F1": clf_dict['IFG_WHO']['f1-score'], 
            "HbA1c_ADA P": clf_dict['HbA1c_ADA']['precision'],
            "HbA1c_ADA R": clf_dict['HbA1c_ADA']['recall'],
            "HbA1c_ADA F1": clf_dict['HbA1c_ADA']['f1-score'],
            "HbA1c_IEC P": clf_dict['HbA1c_IEC']['precision'],
            "HbA1c_IEC R": clf_dict['HbA1c_IEC']['recall'],
            "HbA1c_IEC F1": clf_dict['HbA1c_IEC']['f1-score'],
            "micro F1": clf_dict['micro avg']['f1-score'], 
            "macro F1": clf_dict['macro avg']['f1-score']
        }
    
    trainer = MultiLabelTrainer(
        model = model,
        args = training_args,
        train_dataset = train_dataset,
        eval_dataset = dev_dataset,
        compute_metrics = compute_metrics,
        callbacks = [early_stop],
    )

    trainer.train()

    trainer.evaluate()
    
    return trainer

### BERT-base-uncased (baseline)

In [12]:
# Parameters
STEPS = 25 # change to 50
NUM_LABELS = 5
MODEL_NAME = "bert-base-uncased"
OUTPUT_DIR = "multilabel_bert_base_uncased"
MAX_LENGTH = 512
EARLY_STOP_VAR = 3
LEARNING_RATE = 1e-5
WEIGHT_DECAY = 0.001
NUM_EPOCHS = 25
GRAD_ACCUM_STEPS = 4

TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)

In [13]:
wandb.init(project="Prediabetes Classification",
           config={
               "model": MODEL_NAME,
               "max_length": MAX_LENGTH,
               "epochs": NUM_EPOCHS,
               "learning_rate": LEARNING_RATE,
           })

In [14]:
TRAIN, TEST = stratify_multilabel_text(merged_df, test_size=0.33) # train/test split -> 77/33
TRAIN, DEV = stratify_multilabel_text(TRAIN, test_size=0.33) # train/dev split -> 77/33

train_dataset = preprocessing(TRAIN, TOKENIZER, MAX_LENGTH)
dev_dataset = preprocessing(DEV, TOKENIZER, MAX_LENGTH)
test_dataset = preprocessing(TEST, TOKENIZER, MAX_LENGTH)

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [15]:
%%wandb

bert_model = train_huggingface_model(
    OUTPUT_DIR,
    MODEL_NAME,
    TOKENIZER,
    train_dataset, 
    dev_dataset, 
    NUM_LABELS, 
    NUM_EPOCHS,
    LEARNING_RATE,
    WEIGHT_DECAY,
    EARLY_STOP_VAR, 
    GRAD_ACCUM_STEPS,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Igt p,Igt r,Igt f1,Ifg Ada p,Ifg Ada r,Ifg Ada f1,Ifg Who p,Ifg Who r,Ifg Who f1,Hba1c Ada p,Hba1c Ada r,Hba1c Ada f1,Hba1c Iec p,Hba1c Iec r,Hba1c Iec f1,Micro f1,Macro f1
25,0.607100,0.586430,0.500000,1.000000,0.666667,0.560000,0.875000,0.682927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.533333,0.269919
50,0.581700,0.580416,0.500000,1.000000,0.666667,0.571429,1.000000,0.727273,0.500000,0.846154,0.628571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.635659,0.404502
75,0.552100,0.572754,0.000000,0.000000,0.000000,0.571429,1.000000,0.727273,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.405063,0.145455
100,0.525600,0.565541,0.550000,0.785714,0.647059,0.571429,1.000000,0.727273,0.470588,0.615385,0.533333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.603448,0.381533
125,0.497000,0.563401,0.523810,0.785714,0.628571,0.571429,1.000000,0.727273,0.470588,0.615385,0.533333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.598291,0.377835
150,0.460600,0.566539,0.500000,0.428571,0.461538,0.571429,1.000000,0.727273,0.800000,0.307692,0.444444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.541667,0.326651
175,0.422900,0.559339,0.571429,0.857143,0.685714,0.571429,1.000000,0.727273,0.473684,0.692308,0.562500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.621849,0.395097
200,0.424400,0.557581,0.545455,0.857143,0.666667,0.571429,1.000000,0.727273,0.411765,0.538462,0.466667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.593220,0.372121
225,0.389400,0.556406,0.555556,0.714286,0.625000,0.592593,1.000000,0.744186,0.461538,0.461538,0.461538,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.587156,0.366145
250,0.359700,0.557100,0.555556,0.714286,0.625000,0.592593,1.000000,0.744186,0.461538,0.461538,0.461538,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.587156,0.366145


In [16]:
bert_model.save_model("final_models/multilabel_bert_base_uncased")

In [18]:
predictions = np.array(bert_model.predict(test_dataset))

/tmp/ipykernel_1308253/2408855104.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predictions = np.array(bert_model.predict(test_dataset))


In [19]:
predictions[2]

{'test_loss': 0.5277460813522339,
 'test_IGT P': 0.52,
 'test_IGT R': 0.6190476190476191,
 'test_IGT F1': 0.5652173913043478,
 'test_IFG_ADA P': 0.6153846153846154,
 'test_IFG_ADA R': 1.0,
 'test_IFG_ADA F1': 0.761904761904762,
 'test_IFG_WHO P': 0.6363636363636364,
 'test_IFG_WHO R': 0.7368421052631579,
 'test_IFG_WHO F1': 0.6829268292682926,
 'test_HbA1c_ADA P': 0.0,
 'test_HbA1c_ADA R': 0.0,
 'test_HbA1c_ADA F1': 0.0,
 'test_HbA1c_IEC P': 0.0,
 'test_HbA1c_IEC R': 0.0,
 'test_HbA1c_IEC F1': 0.0,
 'test_micro F1': 0.6257668711656442,
 'test_macro F1': 0.40200979649548046,
 'test_runtime': 0.5583,
 'test_samples_per_second': 78.805,
 'test_steps_per_second': 78.805}

### BigBird Roberta Base (try large if available)

In [7]:
# Parameters
STEPS = 25 # change to 50
NUM_LABELS = 5
#MODEL_NAME = "google/bigbird-roberta-base"
MODEL_NAME = "google/bigbird-roberta-large"
OUTPUT_DIR = "multilabel_bigbird_large"
MAX_LENGTH = 2554
EARLY_STOP_VAR = 3
LEARNING_RATE = 1e-5
WEIGHT_DECAY = 0.001
NUM_EPOCHS = 25
GRAD_ACCUM_STEPS = 4

TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [8]:
wandb.init(project="Prediabetes Classification",
           config={
               "model": MODEL_NAME,
               "max_length": MAX_LENGTH,
               "epochs": NUM_EPOCHS,
               "learning_rate": LEARNING_RATE,
           })

In [16]:
TRAIN, TEST = stratify_multilabel_text(merged_df, test_size=0.33) # train/test split -> 77/33
TRAIN, DEV = stratify_multilabel_text(TRAIN, test_size=0.33) # train/dev split -> 77/33

train_dataset = preprocessing(TRAIN, TOKENIZER, MAX_LENGTH)
dev_dataset = preprocessing(DEV, TOKENIZER, MAX_LENGTH)
test_dataset = preprocessing(TEST, TOKENIZER, MAX_LENGTH)

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [17]:
%%wandb

bigbird_model = train_huggingface_model(
    OUTPUT_DIR,
    MODEL_NAME,
    TOKENIZER,
    train_dataset, 
    dev_dataset, 
    NUM_LABELS, 
    NUM_EPOCHS,
    LEARNING_RATE,
    WEIGHT_DECAY,
    EARLY_STOP_VAR, 
    GRAD_ACCUM_STEPS,
)

Some weights of the model checkpoint at google/bigbird-roberta-large were not used when initializing BigBirdForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForSequenceClassific

Step,Training Loss,Validation Loss,Igt p,Igt r,Igt f1,Ifg Ada p,Ifg Ada r,Ifg Ada f1,Ifg Who p,Ifg Who r,Ifg Who f1,Hba1c Ada p,Hba1c Ada r,Hba1c Ada f1,Hba1c Iec p,Hba1c Iec r,Hba1c Iec f1,Micro f1,Macro f1
25,0.587500,0.570046,0.000000,0.000000,0.000000,0.571429,1.000000,0.727273,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.405063,0.145455
50,0.561900,0.550980,0.538462,1.000000,0.700000,0.571429,1.000000,0.727273,0.520000,1.000000,0.684211,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.661538,0.422297
75,0.506300,0.522499,0.666667,0.285714,0.400000,0.571429,1.000000,0.727273,0.700000,0.538462,0.608696,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.568421,0.347194
100,0.441600,0.511808,0.800000,0.571429,0.666667,0.571429,1.000000,0.727273,0.714286,0.384615,0.500000,1.000000,0.166667,0.285714,0.000000,0.000000,0.000000,0.618557,0.435931
125,0.382400,0.524694,0.578947,0.785714,0.666667,0.428571,0.375000,0.400000,0.500000,0.769231,0.606061,1.000000,0.333333,0.500000,0.000000,0.000000,0.000000,0.547170,0.434545
150,0.344300,0.522961,0.619048,0.928571,0.742857,0.521739,0.750000,0.615385,0.550000,0.846154,0.666667,0.666667,0.333333,0.444444,0.000000,0.000000,0.000000,0.644068,0.493871
175,0.281200,0.509225,0.631579,0.857143,0.727273,0.473684,0.562500,0.514286,0.529412,0.692308,0.600000,0.750000,0.500000,0.600000,0.000000,0.000000,0.000000,0.600000,0.488312
200,0.259000,0.508111,0.650000,0.928571,0.764706,0.523810,0.687500,0.594595,0.600000,0.692308,0.642857,0.800000,0.666667,0.727273,0.000000,0.000000,0.000000,0.660714,0.545886
225,0.223700,0.519682,0.650000,0.928571,0.764706,0.500000,0.625000,0.555556,0.550000,0.846154,0.666667,0.800000,0.666667,0.727273,0.000000,0.000000,0.000000,0.655172,0.542840
250,0.194900,0.527200,0.650000,0.928571,0.764706,0.466667,0.437500,0.451613,0.500000,0.769231,0.606061,0.800000,0.666667,0.727273,0.000000,0.000000,0.000000,0.612613,0.509930


In [18]:
bigbird_model.save_model("final_models/multilabel_bigbird_large")

In [24]:
predictions = np.array(bigbird_model.predict(test_dataset))

/tmp/ipykernel_1152033/2138710282.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predictions = np.array(bigbird_model.predict(test_dataset))


In [30]:
predictions[2]

{'test_loss': 0.5345389246940613,
 'test_IGT P': 0.5151515151515151,
 'test_IGT R': 0.8095238095238095,
 'test_IGT F1': 0.6296296296296297,
 'test_IFG_ADA P': 0.5789473684210527,
 'test_IFG_ADA R': 0.4583333333333333,
 'test_IFG_ADA F1': 0.5116279069767442,
 'test_IFG_WHO P': 0.5,
 'test_IFG_WHO R': 0.6842105263157895,
 'test_IFG_WHO F1': 0.5777777777777778,
 'test_HbA1c_ADA P': 1.0,
 'test_HbA1c_ADA R': 0.7,
 'test_HbA1c_ADA F1': 0.8235294117647058,
 'test_HbA1c_IEC P': 0.0,
 'test_HbA1c_IEC R': 0.0,
 'test_HbA1c_IEC F1': 0.0,
 'test_micro F1': 0.5925925925925926,
 'test_macro F1': 0.5085129452297716,
 'test_runtime': 10.5316,
 'test_samples_per_second': 4.178,
 'test_steps_per_second': 4.178}

### BioLinkBERT-large

In [31]:
# Parameters
STEPS = 25 # change to 50
NUM_LABELS = 5
MODEL_NAME = "michiyasunaga/BioLinkBERT-large"
OUTPUT_DIR = "multilabel_biolinkbert_large"
MAX_LENGTH = 512
EARLY_STOP_VAR = 3
LEARNING_RATE = 1e-5
WEIGHT_DECAY = 0.001
NUM_EPOCHS = 25
GRAD_ACCUM_STEPS = 4

TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)

In [32]:
wandb.init(project="Prediabetes Classification",
           config={
               "model": MODEL_NAME,
               "max_length": MAX_LENGTH,
               "epochs": NUM_EPOCHS,
               "learning_rate": LEARNING_RATE,
           })

eval/HbA1c_ADA F1,▁▁▁▄▆▅▇████████
eval/HbA1c_ADA P,▁▁▁██▆▆▇▇▇▇▇▇▇▇
eval/HbA1c_ADA R,▁▁▁▃▅▅▆████████
eval/HbA1c_IEC F1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/HbA1c_IEC P,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/HbA1c_IEC R,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/IFG_ADA F1,████▁▆▃▅▄▂▂▂▃▂▂
eval/IFG_ADA P,████▁▆▃▆▄▃▄▇▆▇▇
eval/IFG_ADA R,████▁▅▃▅▄▂▂▁▂▁▁
eval/IFG_WHO F1,▁█▇▆▇█▇██▇██▇██
eval/IFG_WHO P,▁▆██▆▆▆▇▆▆▇▆▆▆▆


In [33]:
TRAIN, TEST = stratify_multilabel_text(merged_df, test_size=0.33) # train/test split -> 77/33
TRAIN, DEV = stratify_multilabel_text(TRAIN, test_size=0.33) # train/dev split -> 77/33

train_dataset = preprocessing(TRAIN, TOKENIZER, MAX_LENGTH)
dev_dataset = preprocessing(DEV, TOKENIZER, MAX_LENGTH)
test_dataset = preprocessing(TEST, TOKENIZER, MAX_LENGTH)

Map:   0%|          | 0/57 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [34]:
%%wandb

biolink_model = train_huggingface_model(
    OUTPUT_DIR,
    MODEL_NAME,
    TOKENIZER,
    train_dataset, 
    dev_dataset, 
    NUM_LABELS, 
    NUM_EPOCHS,
    LEARNING_RATE,
    WEIGHT_DECAY,
    EARLY_STOP_VAR, 
    GRAD_ACCUM_STEPS,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at michiyasunaga/BioLinkBERT-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/joemenke/.local/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Igt p,Igt r,Igt f1,Ifg Ada p,Ifg Ada r,Ifg Ada f1,Ifg Who p,Ifg Who r,Ifg Who f1,Hba1c Ada p,Hba1c Ada r,Hba1c Ada f1,Hba1c Iec p,Hba1c Iec r,Hba1c Iec f1,Micro f1,Macro f1
25,0.607800,0.560381,0.000000,0.000000,0.000000,0.586207,1.000000,0.739130,1.000000,0.307692,0.470588,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.494118,0.241944
50,0.527100,0.544645,0.285714,0.142857,0.190476,0.611111,0.647059,0.628571,1.000000,0.076923,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.358974,0.192381
75,0.437600,0.514144,0.600000,0.857143,0.705882,0.666667,0.823529,0.736842,0.600000,0.692308,0.642857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.648148,0.417116
100,0.353900,0.513314,0.555556,0.714286,0.625000,0.619048,0.764706,0.684211,0.562500,0.692308,0.620690,1.000000,0.166667,0.285714,0.000000,0.000000,0.000000,0.611111,0.443123
125,0.271800,0.506632,0.631579,0.857143,0.727273,0.578947,0.647059,0.611111,0.642857,0.692308,0.666667,0.500000,0.166667,0.250000,0.000000,0.000000,0.000000,0.622642,0.451010
150,0.202300,0.516568,0.650000,0.928571,0.764706,0.588235,0.588235,0.588235,0.611111,0.846154,0.709677,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.635514,0.412524
175,0.162000,0.553903,0.560000,1.000000,0.717949,0.636364,0.823529,0.717949,0.578947,0.846154,0.687500,1.000000,0.166667,0.285714,0.000000,0.000000,0.000000,0.672269,0.481822
200,0.126300,0.551696,0.541667,0.928571,0.684211,0.636364,0.823529,0.717949,0.578947,0.846154,0.687500,1.000000,0.500000,0.666667,0.000000,0.000000,0.000000,0.683333,0.551265
225,0.103200,0.550300,0.583333,1.000000,0.736842,0.636364,0.823529,0.717949,0.555556,0.769231,0.645161,1.000000,0.500000,0.666667,0.000000,0.000000,0.000000,0.689076,0.553324
250,0.088300,0.551208,0.565217,0.928571,0.702703,0.636364,0.823529,0.717949,0.625000,0.769231,0.689655,1.000000,0.666667,0.800000,0.000000,0.000000,0.000000,0.700855,0.582061


In [35]:
biolink_model.save_model("final_models/multilabel_biolink_large")

In [36]:
predictions = np.array(biolink_model.predict(test_dataset))
predictions[2]

/tmp/ipykernel_1152033/4168129558.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predictions = np.array(biolink_model.predict(test_dataset))


{'test_loss': 0.5004033446311951,
 'test_IGT P': 0.6129032258064516,
 'test_IGT R': 0.9047619047619048,
 'test_IGT F1': 0.7307692307692307,
 'test_IFG_ADA P': 0.6666666666666666,
 'test_IFG_ADA R': 0.8333333333333334,
 'test_IFG_ADA F1': 0.7407407407407408,
 'test_IFG_WHO P': 0.6521739130434783,
 'test_IFG_WHO R': 0.7894736842105263,
 'test_IFG_WHO F1': 0.7142857142857143,
 'test_HbA1c_ADA P': 1.0,
 'test_HbA1c_ADA R': 0.7,
 'test_HbA1c_ADA F1': 0.8235294117647058,
 'test_HbA1c_IEC P': 0.0,
 'test_HbA1c_IEC R': 0.0,
 'test_HbA1c_IEC F1': 0.0,
 'test_micro F1': 0.7261904761904762,
 'test_macro F1': 0.6018650195120783,
 'test_runtime': 1.9989,
 'test_samples_per_second': 22.012,
 'test_steps_per_second': 22.012}

### BioGPT

In [7]:
# Parameters
NUM_LABELS = 5
MODEL_NAME = "microsoft/biogpt"
MAX_LENGTH = 1024 # default; longest sequence currently = 2554 -> works w/ gradient checkpointing
NUM_EPOCHS = 25
OUTPUT_DIR = "final_models/output_biogpt/state_dict_best_model.pt" # include a directory and a file
SHUFFLE = True
NUM_WORKERS = 0
BATCH_SIZE = 1
LEARNING_RATE = 1e-5
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)

In [8]:
wandb.init(project="Prediabetes Classification",
           config={
               "model": MODEL_NAME,
               "max_length": MAX_LENGTH,
               "epochs": NUM_EPOCHS,
               "learning_rate": LEARNING_RATE,
           })

In [9]:
TRAIN, TEST = stratify_multilabel_text(merged_df, test_size=0.33) # train/test split -> 77/33
TRAIN, DEV = stratify_multilabel_text(TRAIN, test_size=0.33) # train/dev split -> 77/33
print(f"Training size: {TRAIN.shape}; Dev size: {DEV.shape}; Testing size: {TEST.shape}")

train_dataset = preprocessing(TRAIN, TOKENIZER, MAX_LENGTH)
dev_dataset = preprocessing(DEV, TOKENIZER, MAX_LENGTH)
test_dataset = preprocessing(TEST, TOKENIZER, MAX_LENGTH)

train_params = {'batch_size': BATCH_SIZE, 'shuffle': SHUFFLE, 'num_workers': NUM_WORKERS}
dev_params = {'batch_size': BATCH_SIZE, 'shuffle': SHUFFLE, 'num_workers': NUM_WORKERS}
test_params = {'batch_size': BATCH_SIZE, 'shuffle': SHUFFLE, 'num_workers': NUM_WORKERS}

TRAIN_LOADER = DataLoader(train_dataset, **train_params)
DEV_LOADER = DataLoader(dev_dataset, **dev_params)
TEST_LOADER = DataLoader(test_dataset, **test_params)

Training size: (58, 2); Dev size: (28, 2); Testing size: (44, 2)


Map:   0%|          | 0/58 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [10]:
MODEL = BioGptForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification",
    max_position_embeddings=MAX_LENGTH,
).to(DEVICE)

MODEL.gradient_checkpointing_enable()

Some weights of the model checkpoint at microsoft/biogpt were not used when initializing BioGptForSequenceClassification: ['output_projection.weight']
- This IS expected if you are initializing BioGptForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BioGptForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def biogpt_save_model(model, epoch, optimizer, out_file):
    out_dir = "".join(out_file.rsplit('/', 1)[:-1])
    print(f"...Saving model")
    if os.path.exists(out_dir):
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, out_file)
    else:
        os.makedirs(out_dir)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, out_file)

def biogpt_validate(device, model, test_dataloader):
    all_preds = torch.tensor((), device='cpu')
    pred_labels = torch.tensor((), device='cpu')

    model.eval()
    for batch in test_dataloader:
        text = batch.pop('text')
        batch['labels'] = torch.transpose(torch.stack(batch['labels'], dim=0), 0, 1)
        labels = batch.pop('labels').type(torch.LongTensor)
        batch['input_ids'] = torch.transpose(torch.stack(batch['input_ids'], dim=0), 0, 1)
        batch['attention_mask'] = torch.transpose(torch.stack(batch['attention_mask'], dim=0), 0, 1)
        for key, value in batch.items():
            batch[key] = batch[key].to(device)
        with torch.no_grad():
            logits = model(**batch).logits

        predictions = torch.sigmoid(logits) > 0.5
        predictions = predictions.type(torch.LongTensor)
        all_preds = torch.cat((all_preds, predictions))
        pred_labels = torch.cat((pred_labels, labels))
    
    model.train()
    
    return all_preds, pred_labels

In [12]:
def biogpt_train(device, model, output_dir, training_loader, testing_loader, num_labels, num_epochs, learn_rate):
    optimizer = AdamW(model.parameters(), lr=learn_rate)
    
    num_training_steps = num_epochs * len(training_loader)
    
    progress_bar = tqdm(range(num_training_steps))
    best_f1_score_macro = 0
    model.train()
    for epoch in range(num_epochs):
        for idx, batch in enumerate(training_loader):
            optimizer.zero_grad()
            
            batch.pop('text')
            batch['labels'] = torch.transpose(torch.stack(batch['labels'], dim=0), 0, 1)
            labels = batch.pop('labels').to(torch.float).to(device)
            batch['input_ids'] = torch.transpose(torch.stack(batch['input_ids'], dim=0), 0, 1)
            batch['attention_mask'] = torch.transpose(torch.stack(batch['attention_mask'], dim=0), 0, 1)
            for key, value in batch.items():
                batch[key] = batch[key].to(device)
            
            loss = model(**batch, labels=labels).loss
            loss.backward()
            optimizer.step()
            
            # Validation check every 25 instances
            if (idx % 25 == 0):
                outputs, targets = biogpt_validate(device, model, testing_loader)

                precision_macro = metrics.precision_score(targets, outputs, average='macro', zero_division=0)
                recall_macro = metrics.recall_score(targets, outputs, average='macro')
                f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
                f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
                
                all_labels = ['IGT', 'IFG_ADA', 'IFG_WHO', 'HbA1c_ADA', 'HbA1c_IEC']
                clf_dict = metrics.classification_report(targets, outputs, target_names=all_labels,
                                                         zero_division=0, output_dict=True)
                
                wandb.log({
                    "loss": loss.item(), 
                    "Precision": precision_macro,
                    "Recall": recall_macro,
                    "Micro F1": f1_score_micro,
                    "Macro F1": f1_score_macro,
                })
                
                print(f"Step: {epoch}, {idx}, Loss: {loss.item()}")
                print(f"Precision (Macro): {precision_macro}, Recall (Macro): {recall_macro}")
                print(f"F1 Score (Micro): {f1_score_micro}, F1 Score (Macro): {f1_score_macro}")
                print(clf_dict)
                print()
                if f1_score_macro > best_f1_score_macro:
                    biogpt_save_model(model, epoch, optimizer, output_dir)
                    best_f1_score_macro = f1_score_macro
            
            progress_bar.update(1)
    
    return model

In [13]:
%%wandb

biogpt_model = biogpt_train(
    DEVICE,
    MODEL, 
    OUTPUT_DIR, 
    TRAIN_LOADER, 
    DEV_LOADER, 
    NUM_LABELS, 
    NUM_EPOCHS, 
    LEARNING_RATE,
)

  0%|          | 0/1450 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step: 0, 0, Loss: 0.9119990468025208
Precision (Macro): 0.36468864468864465, Recall (Macro): 0.46973443223443223
F1 Score (Micro): 0.45, F1 Score (Macro): 0.3745182353878006
{'IGT': {'precision': 0.6666666666666666, 'recall': 0.5714285714285714, 'f1-score': 0.6153846153846153, 'support': 14}, 'IFG_ADA': {'precision': 0.42857142857142855, 'recall': 0.1875, 'f1-score': 0.26086956521739124, 'support': 16}, 'IFG_WHO': {'precision': 0.46153846153846156, 'recall': 0.9230769230769231, 'f1-score': 0.6153846153846155, 'support': 13}, 'HbA1c_ADA': {'precision': 0.26666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.3809523809523809, 'support': 6}, 'HbA1c_IEC': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'micro avg': {'precision': 0.391304347826087, 'recall': 0.5294117647058824, 'f1-score': 0.45, 'support': 51}, 'macro avg': {'precision': 0.36468864468864465, 'recall': 0.46973443223443223, 'f1-score': 0.3745182353878006, 'support': 51}, 'weighted avg': {'precision'

  2%|▏         | 26/1450 [00:31<26:52,  1.13s/it] 

Step: 0, 25, Loss: 1.150293231010437
Precision (Macro): 0.36444444444444446, Recall (Macro): 0.29079670329670326
F1 Score (Micro): 0.48888888888888893, F1 Score (Macro): 0.28620155038759687
{'IGT': {'precision': 0.6666666666666666, 'recall': 0.2857142857142857, 'f1-score': 0.4, 'support': 14}, 'IFG_ADA': {'precision': 0.5555555555555556, 'recall': 0.9375, 'f1-score': 0.6976744186046512, 'support': 16}, 'IFG_WHO': {'precision': 0.6, 'recall': 0.23076923076923078, 'f1-score': 0.33333333333333337, 'support': 13}, 'HbA1c_ADA': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'HbA1c_IEC': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'micro avg': {'precision': 0.5641025641025641, 'recall': 0.43137254901960786, 'f1-score': 0.48888888888888893, 'support': 51}, 'macro avg': {'precision': 0.36444444444444446, 'recall': 0.29079670329670326, 'f1-score': 0.28620155038759687, 'support': 51}, 'weighted avg': {'precision': 0.510239651416122, 'recall': 0.4313725490

  4%|▎         | 51/1450 [00:43<26:13,  1.12s/it]

...

 98%|█████████▊| 1418/1450 [12:13<00:37,  1.19s/it]

Step: 24, 25, Loss: 0.0023294638376682997
Precision (Macro): 0.3233766233766234, Recall (Macro): 0.34093406593406594
F1 Score (Micro): 0.49484536082474223, F1 Score (Macro): 0.3250544662309368
{'IGT': {'precision': 0.5, 'recall': 0.7142857142857143, 'f1-score': 0.588235294117647, 'support': 14}, 'IFG_ADA': {'precision': 0.5454545454545454, 'recall': 0.375, 'f1-score': 0.4444444444444444, 'support': 16}, 'IFG_WHO': {'precision': 0.5714285714285714, 'recall': 0.6153846153846154, 'f1-score': 0.5925925925925927, 'support': 13}, 'HbA1c_ADA': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'HbA1c_IEC': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'micro avg': {'precision': 0.5217391304347826, 'recall': 0.47058823529411764, 'f1-score': 0.49484536082474223, 'support': 51}, 'macro avg': {'precision': 0.3233766233766234, 'recall': 0.34093406593406594, 'f1-score': 0.3250544662309368, 'support': 51}, 'weighted avg': {'precision': 0.4540361599185128, 'recall':

100%|█████████▉| 1443/1450 [12:24<00:07,  1.13s/it]

Step: 24, 50, Loss: 0.02743689715862274
Precision (Macro): 0.2866666666666667, Recall (Macro): 0.2802197802197802
F1 Score (Micro): 0.4301075268817204, F1 Score (Macro): 0.28095238095238095
{'IGT': {'precision': 0.4, 'recall': 0.2857142857142857, 'f1-score': 0.3333333333333333, 'support': 14}, 'IFG_ADA': {'precision': 0.5, 'recall': 0.5, 'f1-score': 0.5, 'support': 16}, 'IFG_WHO': {'precision': 0.5333333333333333, 'recall': 0.6153846153846154, 'f1-score': 0.5714285714285715, 'support': 13}, 'HbA1c_ADA': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'HbA1c_IEC': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'micro avg': {'precision': 0.47619047619047616, 'recall': 0.39215686274509803, 'f1-score': 0.4301075268817204, 'support': 51}, 'macro avg': {'precision': 0.2866666666666667, 'recall': 0.2802197802197802, 'f1-score': 0.28095238095238095, 'support': 51}, 'weighted avg': {'precision': 0.40261437908496733, 'recall': 0.39215686274509803, 'f1-score':

100%|██████████| 1450/1450 [12:27<00:00,  1.94it/s]


In [28]:
with torch.no_grad():
    all_preds = torch.tensor((), device='cuda:0')
    pred_labels = torch.tensor((), device='cuda:0')
    for idx, batch in enumerate(TEST_LOADER):
        batch.pop('text')
        batch['labels'] = torch.transpose(torch.stack(batch['labels'], dim=0), 0, 1)
        labels = batch.pop('labels').to(torch.float).to(device)
        batch['input_ids'] = torch.transpose(torch.stack(batch['input_ids'], dim=0), 0, 1)
        batch['attention_mask'] = torch.transpose(torch.stack(batch['attention_mask'], dim=0), 0, 1)
        for key, value in batch.items():
            batch[key] = batch[key].to(device)
        logits = biogpt_model(**batch).logits
        predictions = torch.sigmoid(logits) > 0.5
        predictions = predictions.type(torch.LongTensor).cpu().to(device='cuda:0')
        all_preds = torch.cat((all_preds, predictions))
        pred_labels = torch.cat((pred_labels, labels))
        
    all_labels = ['IGT', 'IFG_ADA', 'IFG_WHO', 'HbA1c_ADA', 'HbA1c_IEC']
    clf_dict = metrics.classification_report(pred_labels.cpu(), all_preds.cpu(), target_names=all_labels,
                                             zero_division=0, output_dict=True)
    print(clf_dict)

/home/joemenke/.local/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


{'IGT': {'precision': 0.5, 'recall': 0.38095238095238093, 'f1-score': 0.4324324324324324, 'support': 21}, 'IFG_ADA': {'precision': 0.5, 'recall': 0.5416666666666666, 'f1-score': 0.52, 'support': 24}, 'IFG_WHO': {'precision': 0.631578947368421, 'recall': 0.631578947368421, 'f1-score': 0.631578947368421, 'support': 19}, 'HbA1c_ADA': {'precision': 0.5, 'recall': 0.1, 'f1-score': 0.16666666666666669, 'support': 10}, 'HbA1c_IEC': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3}, 'micro avg': {'precision': 0.5396825396825397, 'recall': 0.44155844155844154, 'f1-score': 0.48571428571428565, 'support': 77}, 'macro avg': {'precision': 0.42631578947368426, 'recall': 0.33083959899749377, 'f1-score': 0.35013560929350407, 'support': 77}, 'weighted avg': {'precision': 0.512987012987013, 'recall': 0.44155844155844154, 'f1-score': 0.45750321750321754, 'support': 77}, 'samples avg': {'precision': 0.45454545454545453, 'recall': 0.5098484848484849, 'f1-score': 0.4503787878787879, 'support'